# <center> PRAPROSES DAN POS-TAGGING HMM-VITERBI </center>

#1. Import Library NLTK & String

In [1]:
import nltk
import string
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SMC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

2

In [2]:
inputPernyataanKebutuhan = input("Enter Pernyataan Kebutuhan  ")
# Split string into words
list_pernyataankebutuhan = inputPernyataanKebutuhan.split(" ")

Enter Pernyataan Kebutuhan   "Sistem dapat menampilkan halaman login yang berisikan username dan password khusus untuk administrator"\n "Sistem menampilkan page login, dengan inputan username dan password" "Sistem menampilkan page login yang dilengkapi tombol untuk login atau batal"


## 3. Sentence Splitting

In [3]:
def open(list_pernyataankebutuhan):
    all_req = []
    req = open(list_pernyataankebutuhan)
    data_req = req.read()
    for i in range(len(data_req)):
        all_req.append(data_req[i])
    req.close()
    return all_req

In [4]:
all_req = print(list_pernyataankebutuhan)

['"Sistem', 'dapat', 'menampilkan', 'halaman', 'login', 'yang', 'berisikan', 'username', 'dan', 'password', 'khusus', 'untuk', 'administrator"\\n', '"Sistem', 'menampilkan', 'page', 'login,', 'dengan', 'inputan', 'username', 'dan', 'password"', '"Sistem', 'menampilkan', 'page', 'login', 'yang', 'dilengkapi', 'tombol', 'untuk', 'login', 'atau', 'batal"']


## 3. Tokenisasi

In [5]:
tokenizationpreProcess = []
for token in all_req:
    tokenizationpreProcess.append(nltk.word_tokenize(token))

TypeError: 'NoneType' object is not iterable

## 4. Remove Punctuation

In [ ]:
tokenization = []
punct = ['.','!','?',':']
for requirements in tokenizationpreProcess:
    tokenizationKalimat=[req for req in requirements if req not in punct]
    tokenization.append(tokenizationKalimat)
print(tokenization)

## Karakteristik Dataset (opsional)

In [ ]:
def bagofWords(req, wordCount):
    if req in wordCount:
        wordCount[req] += 1
    else:
        wordCount[req] = 1
    return wordCount

In [ ]:
wordCount = {}
totalWord = 1
for reqs in tokenization:
    for req in reqs:
        bagofWords(req,wordCount)
        totalWord +=1
print(wordCount)
print(totalWord)

## 5. POSTAGGING HMM-VITERBI

In [ ]:
def frekuensi(korpus):
    tagCount = {}
    tagCount['<start>'] = 0
    wordTag={}
    tagTrans={}
    idxLine=0
    firstWord=0
    
    while idxLine < len(korpus):
        prevTag = '<start>'
        while not korpus[idxLine].startswith('</kalimat'):
            if not korpus[idxLine].startswith('<kalimat'):
                korpusPart = korpus[idxLine].split('\t')
                if korpusPart[1] in tagCount:
                    tagCount[korpusPart[1]] += 1
                else:
                    tagCount[korpusPart[1]] = 1
                    
                if(korpusPart[0],korpusPart[1]) in wordTag:
                    wordTag[(korpusPart[0], korpusPart[1])] +=1
                else:
                    wordTag[(korpusPart[0],korpusPart[1])] = 1
                
                if firstWord == 1:
                    currentTagTrans=('<start>',korpusPart[1])
                    firstWord = 0
                else:
                    currentTagTrans=(prevTag,korpusPart[1])
                
                if currentTagTrans in tagTrans:
                    tagTrans[currentTagTrans] += 1
                else:
                    tagTrans[currentTagTrans] = 1
                prevTag = korpusPart[1]
            
            else:
                tagCount['<start>'] += 1
                firstWord = 1
            
            idxLine = idxLine+1
        
        idxLine = idxLine + 1
        
    return tagCount, wordTag, tagTrans

In [ ]:
def dictProbTransition(tagTrans, tagCount):
    probabilityTrans = {}
    keys = tagTrans.keys()
    for tag in keys:
        if (tag[0], tag[1]) in tagTrans:
            probabilityTrans[(tag[0],tag[1])] = tagTrans[(tag[0], tag[1])]/tagCount[tag[0]]
    return probabilityTrans

In [ ]:
def dictProbEmission(wordTag, tagCount):
    probabilityEmission = {}
    wordKey = []
    keys = wordTag.keys()
    for part in keys:
        if((part[0],part[1])) in wordTag:
            probabilityEmission[(part[0],part[1])] = wordTag[(part[0],part[1])]/tagCount[(part[1])]
    return probabilityEmission

In [ ]:
with open('Hasil Tag\Tag Korpus anotasi.txt', encoding='utf-8') as file:
    korpus = file.readlines()
korpus = [req.strip() for req in korpus]
tagCount, wordTag, tagTrans = frekuensi(korpus)
print(wordTag)
# # print(tagCount)
# # print(tagTrans)

In [ ]:
probabilityTrans = dictProbTransition(tagTrans,tagCount)
probabilityEmission = dictProbEmission(wordTag, tagCount)

print(probabilityTrans)
# print(probabilityEmission)

In [ ]:
modelTag = {}
modelKata = []
for reqs in wordTag:
    if reqs[1] in modelTag:
        modelTag[reqs[1]] += 1
    else:
        modelTag[reqs[1]] = 1
    if reqs[0] not in modelKata:
        modelKata.append(reqs[0])
tagMax = max(modelTag, key=lambda reqs:modelTag[reqs])
print(modelKata)

In [ ]:
def findTag(probabilityEmission, reqWords):
    tags = []
    for word in probabilityEmission:
        if reqWords == word[0]:
            tags.append(word[1])
    return tags

In [ ]:
def viterbi(probabilityTransition, probabilityEmission, reqWords, tagCount, tagMax, modelKata):
    tagSequence = {}
    viterbiMat = {}
    emissionTable = {}
    transitionTable = {}
    tagArray = []
    temp = {}
    
    keysEmission = probabilityEmission.keys()
    keysTrans = probabilityTransition.keys()
    keysCount = tagCount.keys()
    
    for i in range(len(reqWords)):
        if reqWords[i] not in modelKata:
            modelKata.append(reqWords[i])
            probabilityEmission[(reqWords[i],tagMax)] = 1/tagCount[tagMax]
            
        tagArray = list(set(tagArray+findTag(probabilityEmission,reqWords[i])))

    for tag in tagArray:
        if ('<start>',tag) in keysTrans:
                transitionTable[('<start>',tag)] = probabilityTrans[('<start>',tag)]
        else:
            transitionTable[('<start>',tag)] = 0
    
    for tag1 in tagArray:
        for tag2 in tagArray: 
            if (tag1, tag2) in keysTrans:
                transitionTable[(tag1,tag2)] = probabilityTrans[(tag1,tag2)]
            else:
                transitionTable[(tag1,tag2)] = 0
    
    for req in reqWords:
        for tag in tagArray:
            if (req,tag) in probabilityEmission:
                emissionTable[(req,tag)] = probabilityEmission[(req,tag)]
            else:
                emissionTable[(req,tag)] = 0
    
    for i in range(0,len(reqWords)):
        if i==0:
            for tag in tagArray:
                viterbiMat[(reqWords[i],tag)] = emissionTable[(reqWords[i],tag)]*transitionTable[('<start>',tag)]
                tagSequence[(reqWords[i],tag)] = tag
        else:
            for tag1 in tagArray:
                newScore = {}
                for tag2 in tagArray:
                    newScore[(reqWords[i-1],tag2)] = viterbiMat[(reqWords[i-1],tag2)]*transitionTable[(tag2,tag1)]
                max_path = max(newScore, key=lambda k: newScore[k])
                viterbiMat[(reqWords[i],tag1)] = newScore[max_path] * emissionTable[(reqWords[i],tag1)]
                tagSequence[(reqWords[i],tag1)] = max_path
    
    last_word = reqWords[len(reqWords)-1]
    tagg = tagArray[0]
    tags = [tagg]
    maximum = viterbiMat[(last_word,tagg)]
    for i in range(1,len(tagArray)):
        if viterbiMat[(last_word,tagArray[i])] > maximum:
            maximum = viterbiMat[(last_word,tagArray[i])]
            tags = [tagArray[i]]
    for i in range(len(reqWords)-2,-1,-1):
        tags.insert(0,tagSequence[(reqWords[i+1],tags[0])][1])
    
    return tags  

## 6. Hasil Pemberian Tag

In [ ]:
viterbiResult = []
for reqWords in tokenization:
    viterbiReqs = viterbi(probabilityTrans,probabilityEmission,reqWords, tagCount, tagMax, modelKata)
    viterbiResult.append(viterbiReqs)

for i in range(len(viterbiResult)):
    print(i,viterbiResult[i])

## 7. Transformasi Data Hasil Tag ke Dokumen

In [ ]:
saveFile="Hasil Tag/Tag "+inputname+".txt"
with open(saveFile,"w+",encoding='utf-8') as file: 
    for i in range(len(tokenization)):
        file.write("<kalimat id="+str(i)+">"+"\n")
        for j in range(len(tokenization[i])):
            file.write(tokenization[i][j]+"\t"+viterbiResult[i][j])
            file.write("\n")
        file.write("</kalimat id="+str(i)+">"+"\n")
file.close()

In [ ]:
Sistem Informasi Pelayanan Publik